In [5]:
import torch
import torchvision
import cv2
import numpy as np
from pathlib import Path

from boxmot import BotSort

# Load a pre-trained Faster R-CNN model
device = torch.device('cpu')  # Use 'cuda' if you have a GPU
detector = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
detector.eval().to(device)

# Initialize the tracker
tracker = BotSort(
    reid_weights=Path('osnet_x0_25_msmt17.pt'),  # Path to ReID model
    device=device,  # Use CPU for inference
    half=False
)

# Open the video file
# vid = cv2.VideoCapture(0)  # or 'path/to/your.avi
video_path = r'D:\华毅\目标追踪数据集\1_艾维/20240113-104949_rack-5_right_RGB.mp4'
vid = cv2.VideoCapture(video_path)

while True:
    # Capture frame-by-frame
    ret, frame = vid.read()

    # If ret is False, it means we have reached the end of the video
    if not ret:
        break

    # Convert frame to tensor and move to device
    frame_tensor = torchvision.transforms.functional.to_tensor(frame).to(device)

    # Perform detection
    with torch.no_grad():
        detections = detector([frame_tensor])[0]

    # Filter the detections (e.g., based on confidence threshold)
    confidence_threshold = 0.5
    dets = []
    for i, score in enumerate(detections['scores']):
        if score >= confidence_threshold:
            bbox = detections['boxes'][i].cpu().numpy()
            label = detections['labels'][i].item()
            conf = score.item()
            dets.append([*bbox, conf, label])

    # Convert detections to numpy array (N X (x, y, x, y, conf, cls))
    dets = np.array(dets)

    # Update the tracker
    res = tracker.update(dets, frame)  # --> M X (x, y, x, y, id, conf, cls, ind)

    # Plot tracking results on the image
    tracker.plot_results(frame, show_trajectories=True)
    
    cv2.imshow('BoXMOT + Torchvision', frame)

    # Simulate wait for key press to continue, press 'q' to exit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release resources
vid.release()
cv2.destroyAllWindows()

2024-11-19 10:04:19.806 | INFO     | boxmot.utils.torch_utils:select_device:52 - Yolo Tracking v11.0.5 🚀 Python-3.9.20 torch-2.2.2+cu121CPU
2024-11-19 10:04:19.882 | SUCCESS  | boxmot.appearance.reid_model_factory:load_pretrained_weights:165 - Loaded pretrained weights from osnet_x0_25_msmt17.pt


det_count: 2
first_not_match: 2
no track
handle_not_match, new tracker: 2
create new track
conf= 0.5951144695281982 no new tracker
det_count: 2
first_match: 1
first_not_match: 1
no track
handle_not_match, new tracker: 1
create new track
conf= 0.5432690978050232 no new tracker
det_count: 1
first_match: 1
det_count: 2
first_match: 1
first_not_match: 1
no track
handle_not_match, new tracker: 1
create new track
conf= 0.5428214073181152 no new tracker
det_count: 2
first_not_match: 2
first_iou_dist:
[[1.         0.99957108]]
first_emb_dist: 
[[0.20472232 0.1140703 ]]
first_track_not_match: 1
handle_not_match, new tracker: 2
create new track
conf= 0.5505480170249939 no new tracker
det_count: 2
first_match: 1
first_not_match: 1
no track
find after frame:2 6 4
handle_match, new tracker: 1
det_count: 1
first_match: 1
first_track_not_match: 1
det_count: 1
first_not_match: 1
first_iou_dist:
[[0.98736353]
 [1.        ]]
first_emb_dist: 
[[0.23440861]
 [0.11717479]]
first_track_not_match: 2
handle_n


KeyboardInterrupt

